<a href="https://colab.research.google.com/github/kaushaln1/ML_DM_labs/blob/main/DM/DM_HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error,mean_absolute_percentage_error,mean_squared_log_error

from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer ,make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import Ridge
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DM_4/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DM_4/test.csv')

# Preprocessing

In [ ]:
column_map = {}


for column in df_train.columns:
    unique_vals = df_train[column].unique()
    data_type = df_train[column].dtype

    if len(unique_vals) == 2:
        column_map[column] = 'binary'
        continue

    if data_type == 'object' and len(unique_vals) > 2:
        column_map[column] = 'ordinal'
        continue

    if data_type in ['int64', 'float64']:
        column_map[column] = 'numerical'
        continue


In [ ]:
# normalize numerical attributes
numerical_atrributes = []
for key, value in column_map.items():
        if value == 'numerical':
          numerical_atrributes.append(key)
numerical_atrributes.remove('SalePrice')

df_train_norm_filtered = df_train.filter(numerical_atrributes, axis=1)
sale_p = df_train['SalePrice']
df_tb_nor = df_train_norm_filtered.dropna()

d= preprocessing.normalize(df_tb_nor[numerical_atrributes], axis=0) # normalizing all attributes
normalised_train_data = pd.DataFrame(d, columns=df_tb_nor.columns)
normalised_train_data = pd.concat([normalised_train_data,sale_p ] ,axis = 1)


# normalize test data
df_test_norm_filtered = df_test.filter(numerical_atrributes, axis=1)
test_salep = df_test['SalePrice']
df_test_nor = df_test_norm_filtered.dropna()
d= preprocessing.normalize(df_test_nor[numerical_atrributes], axis=0) # normalizing all attributes
normalised_test_data = pd.DataFrame(d, columns=df_test_nor.columns)
normalised_test_data = pd.concat([normalised_test_data,test_salep ] ,axis = 1)



In [ ]:
#classification data
df_train_class = normalised_train_data.copy()
df_train_class['Class'] = normalised_train_data['SalePrice'].apply(lambda x: 1 if x <= 150000 else -1)
df_train_class.drop('SalePrice', axis=1)

df_test_class = normalised_test_data.copy()
df_test_class['Class'] = normalised_test_data['SalePrice'].apply(lambda x: 1 if x <= 150000 else -1)
df_test_class.drop('SalePrice', axis=1)


#fill nan with mean
df_train_class.fillna(df_train_class.mean(), inplace=True)
df_test_class.fillna(df_test_class.mean(), inplace=True)


In [ ]:
X_train = df_train_class.drop(columns=['Class'])
y_train = df_train_class['Class']

X_test = df_test_class.drop(columns=['Class'])
y_test = df_test_class['Class']


In [ ]:
normalised_test_data

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0.000200,0.015492,0.056154,0.039590,0.050746,0.046762,0.052682,0.052327,0.097490,0.081059,...,0.0,0.000000,0.000000,0.0,0.156461,0.0,0.0,0.046495,0.053078,166000
1,0.000401,0.069713,0.054751,0.023488,0.059203,0.046762,0.053759,0.053423,0.050382,0.102025,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.085241,0.053078,206300
2,0.001002,0.139426,0.024568,0.012226,0.050746,0.046762,0.053974,0.053610,0.020153,0.035775,...,0.0,0.018212,0.000000,0.0,0.000000,0.0,0.0,0.046495,0.053078,216500
3,0.001603,0.023238,0.039308,0.029809,0.042288,0.056114,0.051875,0.052139,0.000000,0.000000,...,0.0,0.000000,0.102329,0.0,0.000000,0.0,0.0,0.023247,0.053131,248000
4,0.001803,0.046475,0.067385,0.037621,0.076118,0.046762,0.054055,0.053690,0.038794,0.072953,...,0.0,0.066342,0.000000,0.0,0.000000,0.0,0.0,0.054244,0.053105,148000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107500
456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240000
457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185000
458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266500


# Regression

In [ ]:
logistic_regression = LogisticRegression()

param_grid = {'C': [0.1, 1, 10, 100]}

grid_search_LR = GridSearchCV(logistic_regression, param_grid, cv=5)
grid_search_LR.fit(X_train, y_train)

print("Best hyperparameters for Logistic Regression:", grid_search_LR.best_params_)

print("Best cross-validation error for Logistic Regression:", 1 - grid_search_LR.best_score_)

test_error_LR = 1 - grid_search_LR.score(X_test, y_test)
print("Test error for Logistic Regression:", test_error_LR)

Best hyperparameters for Logistic Regression: {'C': 0.1}
Best cross-validation error for Logistic Regression: 0.42300000000000004
Test error for Logistic Regression: 0.4260869565217391


# SVM

In [ ]:

svm_model = SVC()


param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10]}


grid_search_SVM = GridSearchCV(svm_model, param_grid, cv=5)
grid_search_SVM.fit(X_train, y_train)


print("Best hyperparameters for SVM:", grid_search_SVM.best_params_)


print("Best cross-validation error for SVM:", 1 - grid_search_SVM.best_score_)

test_error_SVM = 1 - grid_search_SVM.score(X_test, y_test)
print("Test error for SVM:", test_error_SVM)

Best hyperparameters for SVM: {'C': 1, 'gamma': 0.1}
Best cross-validation error for SVM: 0.1309999999999999
Test error for SVM: 0.06739130434782614


# Random Forest

In [ ]:
numerical_atrributes

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [ ]:
numerical_atrributes.append('SalePrice')

#classification data
df_train_class_1  = df_train.filter(numerical_atrributes, axis=1)
df_train_class_1['Class'] = df_train_class_1['SalePrice'].apply(lambda x: 1 if x <= 150000 else -1)


df_test_class_1 = df_test.filter(numerical_atrributes, axis=1)
df_test_class_1['Class'] = df_test_class_1['SalePrice'].apply(lambda x: 1 if x <= 150000 else -1)
df_test_class_1.drop('SalePrice', axis=1)


#fill nan with mean
df_train_class_1.fillna(df_train_class.mean(), inplace=True)
df_test_class_1.fillna(df_test_class.mean(), inplace=True)

X_train = df_train_class_1.drop(columns=['Class'])
y_train = df_train_class_1['Class']

X_test = df_test_class_1.drop(columns=['Class'])
y_test = df_test_class_1['Class']

random_forest = RandomForestClassifier()


param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [ 5, 10, 20]}


grid_search_RF = GridSearchCV(random_forest, param_grid, cv=5)
grid_search_RF.fit(X_train, y_train)


print("Best hyperparameters for Random Forest:", grid_search_RF.best_params_)


print("Best cross-validation error for Random Forest:", 1 - grid_search_RF.best_score_)


test_error_RF = 1 - grid_search_RF.score(X_test, y_test)
print("Test error for Random Forest:", test_error_RF)


Best hyperparameters for Random Forest: {'max_depth': 5, 'n_estimators': 200}
Best cross-validation error for Random Forest: 0.0
Test error for Random Forest: 0.0021739130434782483


# Neural Networks with One Hidden Layer

In [ ]:


neural_network = MLPClassifier(max_iter=1000)

param_grid = {'hidden_layer_sizes': [(10,), (50,), (100,)], 'alpha': [0.0001, 0.001, 0.01]}


grid_search_NN = GridSearchCV(neural_network, param_grid, cv=5)
grid_search_NN.fit(X_train, y_train)


print("Best hyperparameters for Neural Network:", grid_search_NN.best_params_)


print("Best cross-validation error for Neural Network:", 1 - grid_search_NN.best_score_)


test_error_NN = 1 - grid_search_NN.score(X_test, y_test)
print("Test error for Neural Network:", test_error_NN)


Best hyperparameters for Neural Network: {'alpha': 0.0001, 'hidden_layer_sizes': (50,)}
Best cross-validation error for Neural Network: 0.42300000000000004
Test error for Neural Network: 0.4260869565217391


In [ ]:
print ("##############-------------Results -------------#####################")
print(" Logistic Regression Results")
print("Best hyperparameters :", grid_search_LR.best_params_)

print("Best cross-validation error :", 1 - grid_search_LR.best_score_)

test_error_LR = 1 - grid_search_LR.score(X_test, y_test)
print("Test error :", test_error_LR)

##############-------------Results -------------#####################
 Logistic Regression Results
Best hyperparameters : {'C': 0.1}
Best cross-validation error : 0.42300000000000004
Test error : 0.4260869565217391


In [ ]:
print ("##############-------------Results -------------#####################")
print(" SVM Results")
print("Best hyperparameters :", grid_search_SVM.best_params_)

print("Best cross-validation error :", 1 - grid_search_SVM.best_score_)

test_error_SVM = 1 - grid_search_SVM.score(X_test, y_test)
print("Test error :", test_error_SVM)

##############-------------Results -------------#####################
 SVM Results
Best hyperparameters : {'C': 1, 'gamma': 0.1}
Best cross-validation error : 0.1309999999999999
Test error : 0.06739130434782614


In [ ]:
print ("##############-------------Results -------------#####################")
print(" Random forest Results")

print("Best hyperparameters : ", grid_search_RF.best_params_)


print("Best cross-validation error :", 1 - grid_search_RF.best_score_)


test_error_RF = 1 - grid_search_RF.score(X_test, y_test)
print("Test error :", test_error_RF)

##############-------------Results -------------#####################
 Random forest Results
Best hyperparameters :  {'max_depth': 5, 'n_estimators': 200}
Best cross-validation error : 0.0
Test error : 0.0021739130434782483


In [ ]:
print ("##############-------------Results -------------#####################")
print(" Neural Network Results")

print("Best hyperparameters :", grid_search_NN.best_params_)


print("Best cross-validation error :", 1 - grid_search_NN.best_score_)


test_error_NN = 1 - grid_search_NN.score(X_test, y_test)
print("Test error : ", test_error_NN)

##############-------------Results -------------#####################
 Neural Network Results
Best hyperparameters : {'alpha': 0.0001, 'hidden_layer_sizes': (50,)}
Best cross-validation error : 0.42300000000000004
Test error :  0.4260869565217391
